In [2]:
import csv
import ast

input_csv = 'C:/Users/emink/OneDrive/Masaüstü/BERT_Fine_Tune_For_ABSA/data/ABSA4restaurants_train.csv'

with open(input_csv, 'r', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    
    count_net0 = 0
    count_neg1 = 0
    count_pos1 = 0
    count_pos2 = 0
    
    for row in reader:
        # Convert the string representation of a list into an actual Python list
        polarities_list = ast.literal_eval(row['Polarities'])
        
        # Count occurrences in this row, then add to running totals
        count_net0 += polarities_list.count(-0)
        count_neg1 += polarities_list.count(-1)
        count_pos1 += polarities_list.count(1)
        count_pos2 += polarities_list.count(2)

print(f"Total 0: {count_net0}")
print(f"Total -1: {count_neg1}")
print(f"Total 1: {count_pos1}")
print(f"Total 2: {count_pos2}")


Total 0: 1018
Total -1: 44824
Total 1: 864
Total 2: 1224


+ Test
+ Total 0: 248
+ Total -1: 19636
+ Total 1: 310
+ Total 2: 1065

+ ABSA5
+ Total 0: 882
+ Total -1: 38477
+ Total 1: 864
+ Total 2: 861

In [6]:
import pandas as pd
import ast

# Read the CSV file
df = pd.read_csv('C:/Users/emink/OneDrive/Masaüstü/BERT_Fine_Tune_For_ABSA/data/restaurants_test.csv')

# Convert the 'Polarities' column from string representation of a list to an actual Python list
df['Polarities'] = df['Polarities'].apply(ast.literal_eval)

# Filter rows that contain a '2' in the polarity list
df_with_2 = df[df['Polarities'].apply(lambda x: 2 in x)]

# Randomly select 1000 rows to delete from those with polarity 2 
# (make sure there are at least 1000 rows)
rows_to_delete = df_with_2.sample(n=500, random_state=42)

# Drop these rows from the dataframe
df.drop(rows_to_delete.index, inplace=True)

# # Filter rows that contain a '2' in the polarity list
# df_with_0 = df[df['Polarities'].apply(lambda x: 0 in x)]

# # Randomly select 1000 rows to delete from those with polarity 2 
# # (make sure there are at least 1000 rows)
# rows_to_delete = df_with_0.sample(n=100, random_state=42)
# df.drop(rows_to_delete.index, inplace=True)

# Save the updated dataframe
df.to_csv('V1restaurants_test.csv', index=False)


In [1]:
import pandas as pd

# Load the dataset
file_path = "restaurants-trial.csv"  # Replace with your file path
df = pd.read_csv(file_path)

# Remove unnecessary columns
columns_to_remove = ['id', 'from', 'to']
df = df.drop(columns=columns_to_remove, errors='ignore')

# Ensure columns are named properly
df.columns = ['Tokens', 'Tags', 'Polarities']

# Check the first few rows
print(df.head())


df.to_csv('testing_2.csv', index=False)

                                              Tokens        Tags Polarities
0  All the appetizers and salads were fabulous, t...  appetizers   positive
1  All the appetizers and salads were fabulous, t...      salads   positive
2  All the appetizers and salads were fabulous, t...       steak   positive
3  All the appetizers and salads were fabulous, t...       pasta   positive
4                         And really large portions.    portions   positive


In [16]:
# Ensure columns are stringified for processing
df['Tokens'] = df['Tokens'].apply(lambda x: str(x))
df['Tags'] = df['Tags'].apply(lambda x: str(x))
df['Polarities'] = df['Polarities'].apply(lambda x: str(x))

# Save processed data to a new file (optional)
df.to_csv("Processed_Restaurants_Train.csv", index=False)

In [18]:
import csv

input_csv = 'Processed_Restaurants_Train.csv'
output_csv = 'output.csv'

with open(input_csv, 'r', encoding='utf-8', newline='') as infile, \
     open(output_csv, 'w', encoding='utf-8', newline='') as outfile:
    
    reader = csv.DictReader(infile)
    
    # Write the header line exactly as is
    outfile.write(','.join(reader.fieldnames) + '\n')
    
    for row in reader:
        tokens = row['Tokens'].strip()
        
        # Strip *all* leading/trailing quotes. 
        # (Loop until there's no symmetrical pair of quotes left.)
        while tokens.startswith('"') and tokens.endswith('"') and len(tokens) > 1:
            tokens = tokens[1:-1].strip()
        
        # Now wrap once in quotes
        tokens = f'"{tokens}"'
        
        # Build the CSV line manually for 3 columns: Tokens,Tags,Polarities
        # Make sure we keep the exact order of columns the same as the header
        out_line = f'{tokens},{row["Tags"]},{row["Polarities"]}\n'
        
        outfile.write(out_line)


In [22]:
from transformers import BertTokenizer
import pandas as pd

# Load the dataset
file_path = "output.csv"  # Replace with your file path
df = pd.read_csv(file_path)

# Remove unnecessary columns
columns_to_remove = ['id', 'from', 'to']
df = df.drop(columns=columns_to_remove, errors='ignore')

# Ensure the correct column names
df.columns = ['Tokens', 'Tags', 'Polarities']

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# Tokenize and save
def tokenize_data(row):
    tokens = eval(row['Tokens'])  # Convert string representation to list
    tags = eval(row['Tags'])
    polarities = eval(row['Polarities'])
    
    tokenized_tokens = []
    tokenized_tags = []
    tokenized_polarities = []

    for token, tag, polarity in zip(tokens, tags, polarities):
        subwords = tokenizer.tokenize(token)
        tokenized_tokens.extend(subwords)
        tokenized_tags.extend([tag] * len(subwords))
        tokenized_polarities.extend([polarity] * len(subwords))

    return tokenized_tokens, tokenized_tags, tokenized_polarities

# Apply tokenization
df['Tokens'], df['Tags'], df['Polarities'] = zip(*df.apply(tokenize_data, axis=1))

# Save the processed data to a new CSV
output_path = "Processed_Restaurants_Train_Tokenized.csv"
df.to_csv(output_path, index=False)
print(f"Tokenized data saved to {output_path}")


SyntaxError: invalid syntax (<string>, line 1)